<a href="https://colab.research.google.com/github/KimGriezmann/Bigcontest_2021/blob/master/working_space_SeongGwan/BigContest_merge_%232.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [385]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import re
from itertools import chain, repeat

# Data Load

In [244]:
def create_df():

  #연도별 데이터 경로
  HTS_dir = '/content/drive/MyDrive/빅콘테스트/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_HTS'
  Player_dir = '/content/drive/MyDrive/빅콘테스트/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수'
  Hitter_dir = '/content/drive/MyDrive/빅콘테스트/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_타자 기본'
  #단일 데이터 경로
  Team_dir = '/content/drive/MyDrive/빅콘테스트/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_팀.csv'
  Schedule_dir = '/content/drive/MyDrive/빅콘테스트/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_경기일정_2021.csv'
  val_dir = '/content/drive/MyDrive/빅콘테스트/02_평가데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_평가데이터.xlsx'
  Crawl_dir = '/content/drive/MyDrive/빅콘테스트/crawling_data.csv'

  year = [2018, 2019, 2020, 2021]

  for i in year:
    globals()['HTS_{}'.format(i)]     = pd.read_csv(HTS_dir + '_' + str(i) + '.csv', encoding='CP949')
    globals()['Player_{}'.format(i)]  = pd.read_csv(Player_dir + '_' + str(i) + '.csv', encoding='CP949')
    globals()['Hitter_{}'.format(i)]  = pd.read_csv(Hitter_dir + '_' + str(i) + '.csv', encoding='CP949')

  global Team_df, Schedule_df, val_df, Crawl_df
  Team_df = pd.read_csv(Team_dir, encoding='CP949')
  Schedule_df = pd.read_csv(Schedule_dir, encoding='CP949')
  val_df = pd.read_excel(val_dir)
  Crawl_df = pd.read_csv(Crawl_dir)

######################

#연도별 데이터 하나로 통합
def concat_df():
  df_list = ['HTS', 'Player', 'Hitter']

#각 칼럼이 모두 같은지 검사 후 통합
  for i in df_list:
    if ((globals()['{}_2018'.format(i)].columns == globals()['{}_2019'.format(i)].columns).all()) and ((globals()['{}_2019'.format(i)].columns == globals()['{}_2020'.format(i)].columns).all()) and \
    ((globals()['{}_2020'.format(i)].columns == globals()['{}_2021'.format(i)].columns).all()):
      globals()['{}_df'.format(i)] = pd.concat( [ globals()['{}_2018'.format(i)], globals()['{}_2019'.format(i)], globals()['{}_2020'.format(i)], globals()['{}_2021'.format(i) ] ] )

#연도별 길이의 합이 통합된 데이터의 길이와 같은지 검사
  for i in df_list:
    if len(globals()['{}_2018'.format(i)]) + len(globals()['{}_2019'.format(i)]) + len(globals()['{}_2020'.format(i)]) + len(globals()['{}_2021'.format(i)]) == len(globals()['{}_df'.format(i)]):
      print('{} is complete!'.format(i))

######################

create_df()
concat_df()

HTS is complete!
Player is complete!
Hitter is complete!


# DATE, INN 칼럼 추가

In [245]:
HTS_df['DATE'] = HTS_df['G_ID'].apply(lambda x: int(x[:8]))
Crawl_df['DATE'] = Crawl_df['day'].apply(lambda x: int(x.replace('-', '')))
Crawl_df['INN'] = Crawl_df['inn'].apply(lambda x: int(x[0]) if len(x) == 2 else int(x[:2]))

HTS_col = ['DATE', 'PCODE', 'INN', 'HIT_VEL', 'HIT_ANG_VER', 'HIT_RESULT', 'PIT_VEL', 'PIT_ID', 'G_ID']
Crawl_col = ['DATE', 'PCODE', 'NAME', 'POSITION', 'INN', 'result', 'before', 'after', 'LEV', 'REa', 'WPs', 'WPe', 'WPa', 'P', 'pitcher', 'batter', 'vs']

HTS_df = HTS_df[HTS_col]
Crawl_df = Crawl_df[Crawl_col]

In [246]:
HTS_df.head()

,DATE,PCODE,INN,HIT_VEL,HIT_ANG_VER,HIT_RESULT,PIT_VEL,PIT_ID,G_ID
0,20180324,62797,1,131.50,42.7,플라이,149.59,180324_140436,20180324HHWO0
1,20180324,76753,1,135.18,9.9,1루타,148.78,180324_140514,20180324HHWO0
2,20180324,71752,1,152.41,2.1,1루타,148.59,180324_140647,20180324HHWO0
3,20180324,62700,1,113.72,13.5,1루타,139.13,180324_140911,20180324HHWO0
4,20180324,68730,2,54.11,16.4,번트안타,134.07,180324_142050,20180324HHWO0


In [247]:
Crawl_df.head()

,DATE,PCODE,NAME,POSITION,INN,result,before,after,LEV,REa,WPs,WPe,WPa,P,pitcher,batter,vs
0,20180327,60100,백창수,외,8,유격수 땅볼,무사 6:9,1사 6:9,0.88,-0.258,93.100,95.3,-0.022,1-0,이민호,백창수,NC
1,20180328,60100,백창수,외,2,볼넷,2사 2루 0:2,"2사 1,2루 0:2",1.10,0.122,71.900,70.7,0.012,2-3,구창모,백창수,NC
2,20180328,60100,백창수,외,4,3루수 땅볼,2사 1:2,이닝종료 1:2,0.51,-0.117,67.400,68.8,-0.013,0-0,구창모,백창수,NC
3,20180328,60100,백창수,외,7,유격수 땅볼 실책,[실책포함],무사 2:2,무사 1루 2:2,1.540,0.398,50.0,44.100,2-3,유원상,백창수,NC
4,20180330,60100,백창수,외,2,백창수/10473 : 2루수 직선타 아웃,"1사 2,3루 1:1","2사 2,3루 1:1",1.57,-0.833,64.100,56.1,-0.080,0-1,김태훈,백창수,SK


# RESULT 통일

In [248]:
delete_list = ['볼넷', '삼진', '몸에 맞는 볼', '고의4구', '스트라이크낫아웃 출루', '타구맞음', '타격방해', '[실책포함]']

HTS_df_clean = HTS_df[HTS_df['HIT_RESULT'].str.contains('번트') == False]
HTS_df_clean = HTS_df_clean[HTS_df_clean['HIT_RESULT'].str.contains('야수선택') == False]

Crawl_df_clean = Crawl_df[Crawl_df['result'].str.contains('번트') == False]
Crawl_df_clean = Crawl_df_clean[Crawl_df_clean['result'].str.contains('야수선택') == False]
Crawl_df_clean = Crawl_df_clean[Crawl_df_clean['result'].isin(delete_list) == False]
Crawl_df_clean = Crawl_df_clean[Crawl_df_clean['result'].str.contains('삼진') == False]

cd_result = Crawl_df_clean['result'].apply(lambda x : x.split(':')[1].strip() if len(x.split(':'))==2 else x.strip())
cd_result = cd_result.apply(lambda x : re.sub('1루수|2루수|3루수|유격수|포수|우익수|중견수|좌익수|투수', "", x).strip())
Crawl_df_clean.loc[:,['result']] = cd_result

In [249]:
ground_list = ['땅볼아웃', '병살타', '삼중살타']
fly_list = ['플라이', '파울플라이', '희생플라이', '인필드플라이']

HTS_df_clean['RESULT_MG'] = HTS_df_clean['HIT_RESULT'].apply(lambda x: '땅볼' if x in ground_list else x)
HTS_df_clean['RESULT_MG'] = HTS_df_clean['RESULT_MG'].apply(lambda x: '플라이' if x in fly_list else x)
HTS_df_clean['RESULT_MG'] = HTS_df_clean['RESULT_MG'].apply(lambda x: '내야안타' if x == '내야안타(1루타)' else x)

HTS_df_clean['RESULT_MG'].value_counts()

플라이     42571
땅볼      29594
1루타     29235
2루타      8146
홈런       4732
직선타      3602
내야안타     1810
3루타       672
Name: RESULT_MG, dtype: int64

In [257]:
ground_list = ['땅볼', '병살타', '땅볼 실책', '병살타 실책', '삼중살']
fly_list = ['뜬공', '파울 뜬공', '희생플라이', '인필드 뜬공', '플라이 실책', '희생플라이 실책']

Crawl_df_clean['RESULT_MG'] = Crawl_df_clean['result'].apply(lambda x: '땅볼' if x in ground_list else x)
Crawl_df_clean['RESULT_MG'] = Crawl_df_clean['RESULT_MG'].apply(lambda x: '플라이' if x in fly_list else x)
Crawl_df_clean['RESULT_MG'] = Crawl_df_clean['RESULT_MG'].apply(lambda x: '1루타' if x == '안타' else x)
Crawl_df_clean['RESULT_MG'] = Crawl_df_clean['RESULT_MG'].apply(lambda x: '홈런' if x == '그라운드 홈런' else x)
Crawl_df_clean['RESULT_MG'] = Crawl_df_clean['RESULT_MG'].apply(lambda x: '직선타' if '직선타' in x else x)

Crawl_df_clean['RESULT_MG'].value_counts()

땅볼      46972
플라이     44918
1루타     31498
2루타      8683
홈런       4956
직선타      3865
내야안타     3159
3루타       701
Name: RESULT_MG, dtype: int64

# 에러 검사

In [250]:
tmp = HTS_df.drop_duplicates(['PCODE'], keep='first')['PCODE'].reset_index(drop=True).to_list()
tmp2 = Crawl_df.drop_duplicates(['PCODE'], keep='first')['PCODE'].reset_index(drop=True).to_list()

tmp3 = []
for i in tmp2:
    if i not in tmp:
        tmp3.append(i)

pitcher_sel_idx = []
for i in range(len(Crawl_df)):
    if Crawl_df['batter'][i] != Crawl_df['NAME'][i]:
        pitcher_sel_idx.append(i)

tmp4 = []
for i in tmp:
    if i not in tmp2:
        tmp4.append(i)

print('크롤링데이터에는 있고, HTS데이터에는 없는 PCODE: {}개\n크롤링데이터에는 없고, HTS데이터에는 있는 PCODE: {}개\nbatter이름이 최종 선택되지 않은 row: {}개'.format(len(tmp3), len(tmp4), len(pitcher_sel_idx)))

tmp5 = Player_df[Player_df['PCODE'].isin(tmp4)].drop_duplicates(['PCODE'], keep='first').reset_index(drop=True)
except_pcode = list(tmp5.PCODE)

print('\n크롤링데이터에 없고, HTS데이터에만 있는 PCODE 리스트 : {}'.format(except_pcode))

del tmp, tmp2, tmp3, tmp4, tmp5

크롤링데이터에는 있고, HTS데이터에는 없는 PCODE: 13개
크롤링데이터에는 없고, HTS데이터에는 있는 PCODE: 4개
batter이름이 최종 선택되지 않은 row: 0개

크롤링데이터에 없고, HTS데이터에만 있는 PCODE 리스트 : [64944, 67539, 77462, 63636]


In [251]:
origin_len = len(HTS_df_clean)

HTS_df_clean = HTS_df_clean[HTS_df_clean['PCODE'].isin(except_pcode) == False].reset_index(drop=True)
print('삭제된 row 개수 : {}개'.format(origin_len - len(HTS_df_clean)))

삭제된 row 개수 : 206개


In [364]:
len(HTS_df_clean), len(Crawl_df_clean)

(120156, 144752)

## 더블헤더 경기 파악(진행중)

In [106]:
HTS_df_new['HEADER'] = HTS_df_new['G_ID'].apply(lambda x: int(x[-1]))
HTS_df_new[HTS_df_new['HEADER'] == 1]

,GYEAR,DATE,PCODE,T_ID,INN,HIT_VEL,HIT_ANG_VER,HIT_RESULT,PIT_VEL,PIT_ID,G_ID,HEADER
33683,2018,20181006,64657,HT,1,102.30,66.7,파울플라이,117.65,181006_150124,20181006HTSK1,1
33684,2018,20181006,72443,HT,1,159.09,18.5,2루타,129.26,181006_150330,20181006HTSK1,1
33685,2018,20181006,79608,HT,1,140.05,-4.7,땅볼아웃,116.76,181006_150504,20181006HTSK1,1
33686,2018,20181006,71837,SK,1,118.07,20.2,1루타,141.35,181006_150737,20181006HTSK1,1
33687,2018,20181006,62895,SK,1,90.37,-15.2,1루타,132.48,181006_150857,20181006HTSK1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
118933,2021,20210630,79402,SS,9,141.51,23.8,플라이,146.68,210630_175935,20210630SSSK1,1
118934,2021,20210630,62415,SS,9,102.82,-6.2,땅볼아웃,132.90,210630_180113,20210630SSSK1,1
118935,2021,20210630,62895,SK,9,119.47,-15.5,땅볼아웃,147.28,210630_180844,20210630SSSK1,1
118936,2021,20210630,71837,SK,9,107.87,47.0,플라이,148.18,210630_181039,20210630SSSK1,1


In [107]:
HTS_df_new['HEADER'].value_counts()

0    117533
1      1550
2      1452
Name: HEADER, dtype: int64

In [108]:
HTS_df_new[HTS_df_new['HEADER'] == 1]

,GYEAR,DATE,PCODE,T_ID,INN,HIT_VEL,HIT_ANG_VER,HIT_RESULT,PIT_VEL,PIT_ID,G_ID,HEADER
33683,2018,20181006,64657,HT,1,102.30,66.7,파울플라이,117.65,181006_150124,20181006HTSK1,1
33684,2018,20181006,72443,HT,1,159.09,18.5,2루타,129.26,181006_150330,20181006HTSK1,1
33685,2018,20181006,79608,HT,1,140.05,-4.7,땅볼아웃,116.76,181006_150504,20181006HTSK1,1
33686,2018,20181006,71837,SK,1,118.07,20.2,1루타,141.35,181006_150737,20181006HTSK1,1
33687,2018,20181006,62895,SK,1,90.37,-15.2,1루타,132.48,181006_150857,20181006HTSK1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
118933,2021,20210630,79402,SS,9,141.51,23.8,플라이,146.68,210630_175935,20210630SSSK1,1
118934,2021,20210630,62415,SS,9,102.82,-6.2,땅볼아웃,132.90,210630_180113,20210630SSSK1,1
118935,2021,20210630,62895,SK,9,119.47,-15.5,땅볼아웃,147.28,210630_180844,20210630SSSK1,1
118936,2021,20210630,71837,SK,9,107.87,47.0,플라이,148.18,210630_181039,20210630SSSK1,1


# Merge 해보고 오류 검출

In [429]:
HTS_dup = HTS_df_clean[HTS_df_clean.duplicated(['DATE', 'INN', 'PCODE', 'RESULT_MG', 'G_ID'], keep=False)].sort_values(['DATE', 'PCODE', 'PIT_ID'])
HTS_dup_idx = HTS_dup.index

HTS_order_list = [1,2] * int(len(HTS_dup)/2)
HTS_dup['order'] = HTS_order_list

zero = np.zeros(len(HTS_df_clean))
HTS_df_clean['order'] = zero

for i in range(len(HTS_dup)):
    idx = HTS_dup_idx[i]
    HTS_df_clean.loc[idx, 'order'] = HTS_order_list[i]

In [430]:
Crawl_dup = Crawl_df_clean[Crawl_df_clean.duplicated(['DATE', 'INN', 'PCODE', 'RESULT_MG'], keep=False)].sort_values(['DATE', 'PCODE'])
Crawl_dup_idx = Crawl_dup.index

Crawl_order_list = [1,2] * int(len(Crawl_dup)/2)
Crawl_dup['order'] = Crawl_order_list

zero = np.zeros(len(Crawl_df_clean))
Crawl_df_clean['order'] = zero

for i in range(len(Crawl_dup)):
    idx = Crawl_dup_idx[i]
    Crawl_df_clean.loc[idx, 'order'] = Crawl_order_list[i]

In [431]:
test_df = pd.merge(HTS_df_clean, Crawl_df_clean, how='left', on=['DATE', 'INN', 'PCODE', 'RESULT_MG', 'order'])

In [432]:
test_df.head()

,DATE,PCODE,INN,HIT_VEL,HIT_ANG_VER,HIT_RESULT,PIT_VEL,PIT_ID,G_ID,RESULT_MG,order,NAME,POSITION,result,before,after,LEV,REa,WPs,WPe,WPa,P,pitcher,batter,vs
0,20180324,62797,1,131.50,42.7,플라이,149.59,180324_140436,20180324HHWO0,플라이,0.0,양성우,외,뜬공,1사 0:0,2사 0:0,0.62,-0.180,52.2,53.8,-0.016,1-2,로저스,양성우,넥센
1,20180324,76753,1,135.18,9.9,1루타,148.78,180324_140514,20180324HHWO0,1루타,0.0,송광민,내,안타,2사 0:0,2사 1루 0:0,0.40,0.134,53.8,52.6,0.012,0-0,로저스,송광민,넥센
2,20180324,71752,1,152.41,2.1,1루타,148.59,180324_140647,20180324HHWO0,1루타,0.0,김태균,내,안타,2사 1루 0:0,"2사 1,2루 0:0",0.79,0.215,52.6,50.6,0.020,0-2,로저스,김태균,넥센
3,20180324,62700,1,113.72,13.5,1루타,139.13,180324_140911,20180324HHWO0,1루타,0.0,하주석,내,안타,"2사 1,2루 0:0","2사 1,2루 1:0",1.62,1.000,50.6,40.7,0.099,2-1,로저스,하주석,넥센
4,20180324,78288,2,161.90,3.5,1루타,145.75,180324_142310,20180324HHWO0,1루타,0.0,최재훈,포,안타,무사 2루 1:0,"무사 1,3루 1:0",1.13,0.715,33.8,27.9,0.059,0-2,로저스,최재훈,넥센


## 결측값 찾기

In [433]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120156 entries, 0 to 120155
Data columns (total 25 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   DATE         120156 non-null  int64  
 1   PCODE        120156 non-null  int64  
 2   INN          120156 non-null  int64  
 3   HIT_VEL      120156 non-null  float64
 4   HIT_ANG_VER  120156 non-null  float64
 5   HIT_RESULT   120156 non-null  object 
 6   PIT_VEL      120156 non-null  float64
 7   PIT_ID       120156 non-null  object 
 8   G_ID         120156 non-null  object 
 9   RESULT_MG    120156 non-null  object 
 10  order        120156 non-null  float64
 11  NAME         119504 non-null  object 
 12  POSITION     119504 non-null  object 
 13  result       119504 non-null  object 
 14  before       119504 non-null  object 
 15  after        119504 non-null  object 
 16  LEV          119504 non-null  object 
 17  REa          119504 non-null  float64
 18  WPs          119504 non-

In [434]:
notnull_df = test_df[test_df['P'].isnull() == False].reset_index(drop=True)
null_df = test_df[test_df['P'].isnull()].reset_index(drop=True)

In [435]:
null_df.PCODE.value_counts()

# 개명한 선수 3명과 20200825 날짜 8-9회 누락

60558    220
62537     44
67304     10
62415     10
76267      9
        ... 
63088      1
64153      1
66203      1
66710      1
73213      1
Name: PCODE, Length: 151, dtype: int64

In [436]:
null_df

,DATE,PCODE,INN,HIT_VEL,HIT_ANG_VER,HIT_RESULT,PIT_VEL,PIT_ID,G_ID,RESULT_MG,order,NAME,POSITION,result,before,after,LEV,REa,WPs,WPe,WPa,P,pitcher,batter,vs
0,20180324,67394,7,138.23,16.1,플라이,121.91,180324_170208,20180324HHWO0,플라이,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20180324,79109,8,64.73,78.0,파울플라이,140.16,180324_162447,20180324LGNC0,플라이,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20180327,76720,4,33.62,-24.3,땅볼아웃,128.98,180327_200852,20180327HHNC0,땅볼,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20180327,63704,7,49.58,39.9,플라이,119.50,180327_210923,20180327KTSK0,플라이,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20180327,71564,4,149.87,12.2,땅볼아웃,149.55,180327_195006,20180327LTOB0,땅볼,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
647,20210630,51463,7,120.74,37.5,플라이,124.36,210630_171712,20210630SSSK1,플라이,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
648,20210630,62415,9,102.82,-6.2,땅볼아웃,132.90,210630_180113,20210630SSSK1,땅볼,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
649,20210630,62404,5,136.69,20.0,플라이,129.23,210630_203301,20210630SSSK2,플라이,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
650,20210630,51463,7,133.20,40.0,플라이,132.56,210630_210048,20210630SSSK2,플라이,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [439]:
HTS_df_clean[ (HTS_df_clean['DATE'] == 20180324) & (HTS_df_clean['PCODE'] == 79109) ]

,DATE,PCODE,INN,HIT_VEL,HIT_ANG_VER,HIT_RESULT,PIT_VEL,PIT_ID,G_ID,RESULT_MG,order
102,20180324,79109,3,136.59,9.8,1루타,149.37,180324_144253,20180324LGNC0,1루타,0.0
124,20180324,79109,8,64.73,78.0,파울플라이,140.16,180324_162447,20180324LGNC0,플라이,0.0


In [442]:
Crawl_df_clean[ (Crawl_df_clean['DATE'] == 20180324) & (Crawl_df_clean['PCODE'] == 79109) ]

,DATE,PCODE,NAME,POSITION,INN,result,before,after,LEV,REa,WPs,WPe,WPa,P,pitcher,batter,vs,RESULT_MG,order
52477,20180324,79109,오지환,내,3,안타,무사 1:0,무사 1루 1:0,0.87,0.398,38.6,35.2,0.034,2-0,왕웨이중,오지환,NC,1루타,0.0


## 잘 병합됐는지 확인

In [443]:
dup_df = notnull_df[notnull_df.duplicated(['DATE', 'PCODE', 'INN', 'RESULT_MG', 'order'], keep=False)].reset_index()

In [444]:
dup_df

,index,DATE,PCODE,INN,HIT_VEL,HIT_ANG_VER,HIT_RESULT,PIT_VEL,PIT_ID,G_ID,RESULT_MG,order,NAME,POSITION,result,before,after,LEV,REa,WPs,WPe,WPa,P,pitcher,batter,vs


In [445]:
Crawl_df_clean[ (Crawl_df_clean['DATE'] == 20180711) & (Crawl_df_clean['PCODE'] == 67394) ]

,DATE,PCODE,NAME,POSITION,INN,result,before,after,LEV,REa,WPs,WPe,WPa,P,pitcher,batter,vs,RESULT_MG,order
21604,20180711,67394,초이스,외,2,뜬공,무사 1루 1:0,1사 1루 1:0,1.33,-0.380,36.2,39.3,-0.031,0-1,샘슨,초이스,한화,플라이,0.0
21605,20180711,67394,초이스,외,4,2루타,무사 4:0,무사 2루 4:0,0.35,0.634,12.0,9.5,0.024,2-2,샘슨,초이스,한화,2루타,1.0
21606,20180711,67394,초이스,외,4,2루타,"1사 1,2루 10:0","1사 2,3루 11:0",0.04,1.496,0.6,0.4,0.002,2-2,장민재,초이스,한화,2루타,2.0
21608,20180711,67394,초이스,외,7,홈런,2사 1루 17:4,2사 19:4,0.00,1.866,0.1,0.1,0.000,1-3,김경태,초이스,한화,홈런,0.0
21609,20180711,67394,초이스,외,9,안타,2사 1루 19:4,"2사 1,2루 19:4",0.00,0.215,0.0,0.0,0.000,0-0,서균,초이스,한화,1루타,0.0


In [446]:
HTS_df_clean[ (HTS_df_clean['DATE'] == 20180711) & (HTS_df_clean['PCODE'] == 67394) ]

,DATE,PCODE,INN,HIT_VEL,HIT_ANG_VER,HIT_RESULT,PIT_VEL,PIT_ID,G_ID,RESULT_MG,order
20670,20180711,67394,2,153.09,22.2,플라이,140.38,180711_185320,20180711WOHH0,플라이,0.0
20688,20180711,67394,4,144.94,8.5,2루타,123.87,180711_195957,20180711WOHH0,2루타,0.0
20710,20180711,67394,7,162.65,45.4,홈런,140.12,180711_211801,20180711WOHH0,홈런,0.0
20720,20180711,67394,9,174.41,5.8,1루타,138.39,180711_215309,20180711WOHH0,1루타,0.0


In [450]:
notnull_df[ (notnull_df['DATE'] == 20180711) & (notnull_df['PCODE'] == 67394) ]

,DATE,PCODE,INN,HIT_VEL,HIT_ANG_VER,HIT_RESULT,PIT_VEL,PIT_ID,G_ID,RESULT_MG,order,NAME,POSITION,result,before,after,LEV,REa,WPs,WPe,WPa,P,pitcher,batter,vs
20514,20180711,67394,2,153.09,22.2,플라이,140.38,180711_185320,20180711WOHH0,플라이,0.0,초이스,외,뜬공,무사 1루 1:0,1사 1루 1:0,1.33,-0.380,36.2,39.3,-0.031,0-1,샘슨,초이스,한화
20553,20180711,67394,7,162.65,45.4,홈런,140.12,180711_211801,20180711WOHH0,홈런,0.0,초이스,외,홈런,2사 1루 17:4,2사 19:4,0.00,1.866,0.1,0.1,0.000,1-3,김경태,초이스,한화
20563,20180711,67394,9,174.41,5.8,1루타,138.39,180711_215309,20180711WOHH0,1루타,0.0,초이스,외,안타,2사 1루 19:4,"2사 1,2루 19:4",0.00,0.215,0.0,0.0,0.000,0-0,서균,초이스,한화


In [447]:
Crawl_df_clean[(Crawl_df_clean['DATE'] == 20180325) & (Crawl_df_clean['PCODE'] == 62907)]

,DATE,PCODE,NAME,POSITION,INN,result,before,after,LEV,REa,WPs,WPe,WPa,P,pitcher,batter,vs,RESULT_MG,order
9848,20180325,62907,박민우,내,1,뜬공,무사 0:0,1사 0:0,0.87,-0.258,54.800,52.6,-0.022,2-3,김대현,박민우,LG,플라이,0.0
9849,20180325,62907,박민우,내,3,안타,무사 0:0,무사 1루 0:0,0.99,0.398,55.500,59.5,0.040,2-2,김대현,박민우,LG,1루타,0.0
9850,20180325,62907,박민우,내,5,땅볼 실책,[실책포함],무사 0:1,무사 1루 0:1,0.840,0.398,71.0,74.300,1-1,김대현,박민우,LG,땅볼,1.0
9851,20180325,62907,박민우,내,5,땅볼,"2사 1,2루 0:6",이닝종료 0:6,0.14,-0.466,97.800,97.5,-0.004,1-0,최성훈,박민우,LG,땅볼,2.0


In [448]:
HTS_df_clean[(HTS_df_clean['DATE'] == 20180325) & (HTS_df_clean['PCODE'] == 62907)]

,DATE,PCODE,INN,HIT_VEL,HIT_ANG_VER,HIT_RESULT,PIT_VEL,PIT_ID,G_ID,RESULT_MG,order
320,20180325,62907,1,110.50,50.0,플라이,144.33,180325_140829,20180325LGNC0,플라이,0.0
329,20180325,62907,3,105.26,16.9,1루타,143.62,180325_145255,20180325LGNC0,1루타,0.0
335,20180325,62907,5,153.80,-0.2,땅볼아웃,142.03,180325_154714,20180325LGNC0,땅볼,1.0
341,20180325,62907,5,148.16,-7.0,땅볼아웃,138.15,180325_161332,20180325LGNC0,땅볼,2.0


In [449]:
notnull_df[(notnull_df['DATE'] == 20180325) & (notnull_df['PCODE'] == 62907)]

,DATE,PCODE,INN,HIT_VEL,HIT_ANG_VER,HIT_RESULT,PIT_VEL,PIT_ID,G_ID,RESULT_MG,order,NAME,POSITION,result,before,after,LEV,REa,WPs,WPe,WPa,P,pitcher,batter,vs
318,20180325,62907,1,110.50,50.0,플라이,144.33,180325_140829,20180325LGNC0,플라이,0.0,박민우,내,뜬공,무사 0:0,1사 0:0,0.87,-0.258,54.800,52.6,-0.022,2-3,김대현,박민우,LG
327,20180325,62907,3,105.26,16.9,1루타,143.62,180325_145255,20180325LGNC0,1루타,0.0,박민우,내,안타,무사 0:0,무사 1루 0:0,0.99,0.398,55.500,59.5,0.040,2-2,김대현,박민우,LG
333,20180325,62907,5,153.80,-0.2,땅볼아웃,142.03,180325_154714,20180325LGNC0,땅볼,1.0,박민우,내,땅볼 실책,[실책포함],무사 0:1,무사 1루 0:1,0.840,0.398,71.0,74.300,1-1,김대현,박민우,LG
339,20180325,62907,5,148.16,-7.0,땅볼아웃,138.15,180325_161332,20180325LGNC0,땅볼,2.0,박민우,내,땅볼,"2사 1,2루 0:6",이닝종료 0:6,0.14,-0.466,97.800,97.5,-0.004,1-0,최성훈,박민우,LG
